# Preprocessing

This notebook shows preprocessing of the dataset.

In [ ]:
# Import packages
import pandas as pd
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
from datasets import Dataset, Audio
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import torch
# import torchaudio
# from datetime import datetime
# import librosa
import re

In [2]:
# Load model and processor
model_name = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Load training data
train_df = pd.read_csv('training_data.csv')

In [4]:
# Initialize data collator
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

## Preprocessing Data

In [5]:
# Ensure dataset vocab is in Wav2Vec2 processor
vocab = processor.tokenizer.get_vocab()
allowed_chars = set(''.join(vocab.keys()))


def clean_text(text):
    '''
    Clean transcriptions. Ensures that dataset transcriptions are in Wav2Vec2 tokenizer vocab.
    '''
    text = text.upper() #Wav2Vec2 vocab is upper case
    text = ''.join([c if c in allowed_chars else '' for c in text])
    text = re.sub(r"\s+", "", text).strip()
    return text



def preprocess(example):
    '''
    Preprocess audio files for training.
    '''
    try:
        # Load audio
        waveform, sr = torchaudio.load(example["file_name"])
            
        # Resample
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            waveform = resampler(waveform)

        # Flatten list of waveforms for correct dimensionality
        waveform = waveform.squeeze(0)  # shape becomes [samples]

        # Use the feature extractor to encode audio
        inputs = processor.feature_extractor(
            waveform,
            sampling_rate=16000,
            padding=True,
            return_attention_mask=True,
            return_tensors="pt"  
            )

        # Remove the batch dimension
        input_values = inputs["input_values"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)

        # Tokenize word (label_ids)
        with processor.as_target_processor():
            labels = processor.tokenizer(
                example["clean_text"].upper(),
                return_tensors="pt",
                padding=True,
                truncation=True
                ).input_ids.squeeze(0) 

        # Remove files that were not processed correctly
        if all(label == -100 for label in labels):
            return None
        
        return {
            "input_values": input_values,
            "attention_mask": attention_mask,
            "labels": labels
            }

    except Exception as e:
        print(f"Skipping example due to error: {e}")
        return None


def is_valid(example):
    '''
    Additional pass to ensure files are valid for training.
    '''
    return any(label != -100 for label in example['labels'])

In [ ]:
# Process dataset
train_df['clean_text'] = train_df['transcription'].apply(lambda x: clean_text(x))
dataset = Dataset.from_pandas(train_df)
dataset = dataset.map(preprocess)
filtered_dataset=dataset.filter(is_valid)
filtered_dataset.save

Map:   0%|          | 0/1192 [00:00<?, ? examples/s]/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Filter: 100%|██████████| 1192/1192 [00:13<00:00, 87.43 examples/s]


In [7]:
filtered_dataset.save_to_disk("training_data.hf")

Saving the dataset (1/1 shards): 100%|██████████| 1192/1192 [00:00<00:00, 4830.00 examples/s]


# Training

In [ ]:
def train_w2v2(lr, output_dir):
    '''
    Fine-tune on dataset.
    '''
    # Declare training arguments
    training_args = TrainingArguments(
        output_dir = output_dir,
        per_device_train_batch_size=5, 
        per_device_eval_batch_size=5,
        eval_strategy="no",
        num_train_epochs=1,
        learning_rate=lr,
        save_steps=500,
        save_total_limit=2,
        fp16=True,
        eval_steps=500,
        logging_steps=10,
        warmup_steps=100,
        label_names=['labels']
    )

    # Set up trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=filtered_dataset,
        data_collator=DataCollatorCTCWithPadding(processor=processor, padding=True),
        tokenizer=processor.feature_extractor
    )

    # Train
    trainer.train()

    # Save model
    model.save_pretrained(output_dir)
    processor.save_pretrained(output_dir)

In [ ]:
train_w2v2(1e-6, "./w2v2960h_lr1e6")

# Run ASR

In [ ]:
# Turn into dictionary of models
base = "facebook/wav2vec2-base-960h"
lr0 = './w2v2960h_lr0'
lr1e4 = './w2v2960h_lr1e4'
lr1e8 = "./w2v2960h_lr1e8"
lr1e16 = "./w2v2960h_lr1e16"

# Turn this into a function
base_model = Wav2Vec2ForCTC.from_pretrained(base)
base_proc = Wav2Vec2Processor.from_pretrained(base)

lr0_model = Wav2Vec2ForCTC.from_pretrained(lr0)
lr0_proc = Wav2Vec2Processor.from_pretrained(lr0)

lr1e4_model = Wav2Vec2ForCTC.from_pretrained(lr1e4)
lr1e4_proc = Wav2Vec2Processor.from_pretrained(lr1e4)

lr1e8_model = Wav2Vec2ForCTC.from_pretrained(lr1e8)
lr1e8_proc = Wav2Vec2Processor.from_pretrained(lr1e8)

lr1e16_model = Wav2Vec2ForCTC.from_pretrained(lr1e16)
lr1e16_proc = Wav2Vec2Processor.from_pretrained(lr1e16)

In [ ]:
df = pd.read_csv("./eval.csv")
df =df.head(30)
df

In [ ]:
def load_audio(file_path):
    '''
    Load audio file from filepath.
    '''
    audio, _ = librosa.load(file_path, sr=16000)
    return audio


def transcribe_audio(model, proc, file_name):
    '''
    Transcribes audio using defined model.
    '''
    audio = load_audio(file_name)
    
    input_vals = proc(
        audio, 
        return_tensors = 'pt',
        sampling_rate = 16000
    ).input_values 

    with torch.no_grad():
        logits = model(input_vals).logits 
        
    predicted_ids = torch.argmax(logits, dim = -1)
    predicted_word = proc.decode(predicted_ids[0])
    
    return predicted_word.lower()


In [ ]:
# Run base
df["base_pred"] = df["file_name"].apply(lambda x: transcribe_audio(base_model, base_proc, x))
df["lr0_pred"] = df["file_name"].apply(lambda x: transcribe_audio(lr0_model, lr0_proc, x))
df["lr1e16_pred"] = df["file_name"].apply(lambda x: transcribe_audio(lr1e16_model, lr1e16_proc, x))
df["lr1e8_pred"] = df["file_name"].apply(lambda x: transcribe_audio(lr1e8_model, lr1e8_proc, x))
df["lr1e4_pred"] = df["file_name"].apply(lambda x: transcribe_audio(lr1e4_model, lr1e4_proc, x))


In [ ]:
df